## Condition Factory

create a bunch of Condition objects from a csv file and save them to a folder and create Bundle objects from the conditions and save them to a folder

1. Load Condition Template
1. get csv in form

|Condition #|_id|_datetime|_code|_display|
|---|---|---|---|---|
|Foo|foo|2023-08-21T20:16:25.019Z|12345678|Examplitis|

4. Convert csv to DataFrame
<!-- 2. Create a transaction Bundle
1. Load Patient into Bundle
2. Load the data row into the Condition template
3. Add to the Bundler -->
5. Create Condition Instance
5. Validate

### Define Globals and Imports

In [1]:
    import pandas as pd
    from requests import get, post
    from pathlib import Path
    from json import dumps, loads


    in_path = Path.cwd() / 'source_data' /'conditions.csv'
    out_path = Path.cwd() /'examples'
    print(in_path)
    print(out_path)
    _system = 'http://snomed.info/sct'
    fhir_test_server = 'https://r4.ontoserver.csiro.au/fhir'
    # fhir_test_server = 'http://hapi.fhir.org/baseR4'
    #fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
    
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json',
    }
    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 


/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/resource_maker/source_data/conditions.csv
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/resource_maker/examples


### Load Condition Template

In [2]:
%pwd
from my_condition import get_condition_resource as get_condition

my_condition = get_condition('My_Title','my_id','my_datetime','my_code','my_status','my_display')
my_condition['id']

'my_id'

###  Get CSV File ( hard coded for now) and Convert to DF

In [3]:
from io import StringIO  # for hardcoded csv only
my_csv = '''_title,_id,_datetime,_code,_status

Examplitis,examplitis,2023-08-21T20:16:25.019Z,123456789,active
Ear Burn,ear_burn,2021-01-21,39065001,resolved
Abscess,abscess,2018-02-02,18099001,resolved
COPD,copd,2023-08-22,13645005,active
'''

df = pd.read_csv(StringIO(my_csv), sep=",", dtype='str')

df

,_title,_id,_datetime,_code,_status
0,Examplitis,examplitis,2023-08-21T20:16:25.019Z,123456789,active
1,Ear Burn,ear_burn,2021-01-21,39065001,resolved
2,Abscess,abscess,2018-02-02,18099001,resolved
3,COPD,copd,2023-08-22,13645005,active


### Create Condition Instance

1. Get Data Element row by row from DF

In [4]:
def get_display(_system,_code,_version=None):
    """for example Looking up a code/system
       GET [base]/CodeSystem/$lookup?system=http://loinc.org&code=1963-8
    """
    params = dict(
       system = _system,
      # system = 'http://loinc.org',
      code = _code,
      # code = '1963-8'
        )
    r = get(f'{fhir_test_server}/CodeSystem/$lookup', params = params, headers = headers)
    print(r.status_code, r.reason)
    print(r.json())
    if r.status_code != 200:
        return 'examplitis'
    else:
      return r.json()['parameter'][1]['valueString']

In [5]:
for i in range(len(df)):
    # print(i, df.iloc[i])
    _title,_id,_datetime,_code,_status = df.iloc[i]
    _display = get_display(_system,_code)
    my_condition = get_condition(_title,_id,_datetime,_code,_status,_display)
    # print(my_condition)
    out_file = out_path / f'{_title}.json'
    out_file.write_text(dumps(my_condition, indent=2))


404 Not Found
{'resourceType': 'OperationOutcome', 'issue': [{'severity': 'error', 'code': 'not-found', 'details': {'text': '[940f3ce8-5fb3-42bc-b9ca-9301309fe59f]: The code 123456789 (http://snomed.info/sct:http://snomed.info/sct/32506021000036107/version/20230731) was not found.'}, 'diagnostics': '[940f3ce8-5fb3-42bc-b9ca-9301309fe59f]: The code 123456789 (http://snomed.info/sct:http://snomed.info/sct/32506021000036107/version/20230731) was not found.'}]}
200 OK
{'resourceType': 'Parameters', 'parameter': [{'name': 'code', 'valueCode': '39065001'}, {'name': 'display', 'valueString': 'Burn of ear'}, {'name': 'name', 'valueString': 'SNOMED Clinical Terms Australian extension'}, {'name': 'system', 'valueUri': 'http://snomed.info/sct'}, {'name': 'version', 'valueString': 'http://snomed.info/sct/32506021000036107/version/20230731'}, {'name': 'property', 'part': [{'name': 'code', 'valueCode': 'parent'}, {'name': 'value', 'valueCode': '262581006'}]}, {'name': 'property', 'part': [{'name': '